In [1]:
from getpass import getpass
import os

if "OPENAI_APIKEY" not in os.environ:
    os.environ["OPENAI_APIKEY"] = getpass("Enter your OpenAI API Key")

In [2]:
import weaviate
import weaviate.classes.config as wc
from weaviate.util import generate_uuid5
import os
from tqdm import tqdm
import pandas as pd

headers = {"X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY")}
client = weaviate.connect_to_local(headers=headers)

if client.collections.exists("Games"):
    client.collections.delete("Games")
client.collections.create(
    name="Games",
    properties=[
        wc.Property(name="name", data_type=wc.DataType.TEXT),
        wc.Property(name="price", data_type=wc.DataType.INT),
        wc.Property(name="platforms", data_type=wc.DataType.TEXT_ARRAY),
        wc.Property(name="release_date", data_type=wc.DataType.DATE),
        wc.Property(name="description", data_type=wc.DataType.TEXT),
    ],
    generative_config=wc.Configure.Generative.ollama(model="deepseek-r1:1.5b",
                                                     api_endpoint="http://host.docker.internal:11434"),
    vectorizer_config=wc.Configure.Vectorizer.text2vec_openai(),

)

In [3]:

games = client.collections.get("Games")

df = pd.read_csv('games.csv')

with games.batch.dynamic() as batch:
    for i, game in tqdm(df.iterrows()):
        platforms = game["platform"].split(',') if type(game["platform"]) is str else []
        game_obj = {
            "name": game["name"],
            "platforms": platforms,
            "price": game["price"],
            "release_date": game["release_date"],
            "description": game["description"],
        }

        batch.add_object(
            properties=game_obj,
            uuid=generate_uuid5(game["id"])
        )
if len(games.batch.failed_objects) > 0:
    print(f"Failed to import {len(games.batch.failed_objects)} objects")
    print(games.batch.failed_objects)

915it [00:05, 176.23it/s]


In [4]:
response = games.query.near_text(query="I play the vilain", limit=3)

for o in response.objects:
    print(o.properties)

{'platforms': ['Windows'], 'description': "A dark fantasy roguelike where you play as the Devil! Lead famous evil geniuses through events and turn-based fights to spread terror and corruption, and use your evil powers to change the game's rules to your advantage.", 'price': 2499, 'release_date': datetime.datetime(2021, 9, 30, 8, 0, tzinfo=datetime.timezone.utc), 'name': 'Rogue Lords'}
{'platforms': ['Windows'], 'description': 'Smash, clobber and bash the murderous legends of Slavic mythology in this darkly funny action role-playing game that changes every time you play. Play as Ivan, a one-handed blacksmith with incredibly bad luck, who must take on the impossible tasks given to him by the tzar. All...', 'price': 2499, 'release_date': datetime.datetime(2019, 11, 12, 14, 0, tzinfo=datetime.timezone.utc), 'name': 'Yaga'}
{'platforms': ['Windows'], 'description': 'In a violent, medieval world, outplay rival gangs in intense PvPvE multiplayer heists. Moving in stealth to steal treasures un

In [8]:
games.config.get().to_dict()

{'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
  'cleanupIntervalSeconds': 60,
  'indexNullState': False,
  'indexPropertyLength': False,
  'indexTimestamps': False,
  'stopwords': {'preset': 'en'}},
 'multiTenancyConfig': {'enabled': False,
  'autoTenantCreation': False,
  'autoTenantActivation': False},
 'properties': [{'name': 'name',
   'dataType': ['text'],
   'indexFilterable': True,
   'indexSearchable': True,
   'indexRangeFilters': False,
   'tokenization': 'word',
   'moduleConfig': {'text2vec-openai': {'skip': False,
     'vectorizePropertyName': True}}},
  {'name': 'price',
   'dataType': ['int'],
   'indexFilterable': True,
   'indexSearchable': False,
   'indexRangeFilters': False,
   'tokenization': None,
   'moduleConfig': {'text2vec-openai': {'skip': False,
     'vectorizePropertyName': True}}},
  {'name': 'platforms',
   'dataType': ['text[]'],
   'indexFilterable': True,
   'indexSearchable': True,
   'indexRangeFilters': False,
   'tokenization': 'word',
 

In [ ]:
response = games.generate.near_text(
    query="What are some games where I get to play the vilain?",
    limit=5,
    grouped_task="Provide an answer to the question by making sure you refer to the price for each game as well as the platforms it's available on.",
    grouped_properties=["name", "description", "price", "platforms"],     
)

print(response.generated)
for o in response.objects:
    print(o.properties)
    print(o.generated)

WeaviateQueryError: Query call with protocol GRPC search failed with message <AioRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "explorer: get class: concurrentTargetVectorSearch): explorer: get class: extend: extend generate: client not found, empty provider"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2025-01-31T14:01:42.322482+01:00", grpc_status:2, grpc_message:"explorer: get class: concurrentTargetVectorSearch): explorer: get class: extend: extend generate: client not found, empty provider"}"
>.

In [8]:
import ollama

output = ollama.generate(
  model="deepseek-r1:1.5b",
  prompt="What is the capital of china?"
)

print(output['response'])

<think>

</think>

The capital of China is Beijing.
